# SVM

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics, model_selection
import random

%matplotlib inline

## Load the data

In [2]:
data = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

### Handling datetime
* spilt datetime into day, month and year for training

In [3]:
new = data["Date"].str.split("-", n = 2, expand = True) 

new.columns = ['Day', 'Month', 'Year']
new.loc[new['Month'] == 'Jan', ['Month']] = 1
new.loc[new['Month'] == 'Feb', ['Month']] = 2
new.loc[new['Month'] == 'Mar', ['Month']] = 3
new.loc[new['Month'] == 'Apr', ['Month']] = 4
new.loc[new['Month'] == 'May', ['Month']] = 5
new.loc[new['Month'] == 'Jun', ['Month']] = 6
new.loc[new['Month'] == 'Jul', ['Month']] = 7
new.loc[new['Month'] == 'Aug', ['Month']] = 8
new.loc[new['Month'] == 'Sep', ['Month']] = 9
new.loc[new['Month'] == 'Oct', ['Month']] = 10
new.loc[new['Month'] == 'Nov', ['Month']] = 11
new.loc[new['Month'] == 'Dec', ['Month']] = 12

data = data.drop('Date', axis=1)
data = pd.concat([new, data], axis=1)

new2 = test["Date"].str.split("-", n = 2, expand = True) 

new2.columns = ['Day', 'Month', 'Year']
new2.loc[new2['Month'] == 'Jan', ['Month']] = 1
new2.loc[new2['Month'] == 'Feb', ['Month']] = 2
new2.loc[new2['Month'] == 'Mar', ['Month']] = 3
new2.loc[new2['Month'] == 'Apr', ['Month']] = 4
new2.loc[new2['Month'] == 'May', ['Month']] = 5
new2.loc[new2['Month'] == 'Jun', ['Month']] = 6
new2.loc[new2['Month'] == 'Jul', ['Month']] = 7
new2.loc[new2['Month'] == 'Aug', ['Month']] = 8
new2.loc[new2['Month'] == 'Sep', ['Month']] = 9
new2.loc[new2['Month'] == 'Oct', ['Month']] = 10
new2.loc[new2['Month'] == 'Nov', ['Month']] = 11
new2.loc[new2['Month'] == 'Dec', ['Month']] = 12

test = test.drop('Date', axis=1)
test = pd.concat([new2, test], axis=1)

### Specifying the ups and downs
* Subtract close price to open price.
* If the result is positive(up), remark it as 1
* Otherwise, remark it as 0
* Up shift the row to predict the ups and downs tomorrow

In [4]:
diff = pd.DataFrame(data['Close Price'] - data['Open Price'])
diff.columns = ['Diff']
diff = diff.drop(diff.index[0]).reset_index(drop=True)

data = pd.concat([data, diff], axis=1)
data.loc[data['Diff'] >= 0, ['Diff']] = 1
data.loc[data['Diff'] < 0, ['Diff']] = 0
data = data.fillna(0)

diff2 = pd.DataFrame(test['Close Price'] - test['Open Price'])
diff2.columns = ['Diff']
diff2 = diff2.drop(diff2.index[0]).reset_index(drop=True)

test = pd.concat([test, diff2], axis=1)
test.loc[test['Diff'] >= 0, ['Diff']] = 1
test.loc[test['Diff'] < 0, ['Diff']] = 0
test = test.fillna(0)

In [5]:
data = data.drop(['Volume'], axis=1)
test = test.drop(['Volume'], axis=1)

In [6]:
x_train = data.loc[:, data.columns != 'Diff']
y_train = data.loc[:, data.columns == 'Diff']
x_test = test.loc[:, test.columns != 'Diff']
y_test = test.loc[:, test.columns == 'Diff']

## Standardization

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

X_train_std = scaler.transform(x_train)
X_test_std = scaler.transform(x_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  import sys


## Import svm model in the sklearn

In [8]:
from sklearn.svm import SVC

In [9]:
from sklearn.metrics import accuracy_score

# Custom kernel function
def my_kernel(X, Y):
    return np.dot(X, Y.T)
  
penalty = 0.5

svm = SVC(C=penalty, kernel="linear", probability=True)
svm.fit(x_train, y_train['Diff'].values)
  
acc_rate = accuracy_score(y_test['Diff'].values, svm.predict(x_test)) * 100
print("Penalty = %.2f, Accuracy = %.2f %%" % (penalty, acc_rate))

Penalty = 0.50, Accuracy = 51.59 %
